# Chat Engine with CMEMRetriever (streaming)

To use the CMEM integration, use a RetrieverQueryEngine with CMEMRetriever and CMEMGraphStore.

In [1]:
%pip install cmem-cmempy llama-index python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Load environment from `.env` file. 
Start by `cp .env-template .env` and edit the content of `.env` accordingly.

In [2]:
%load_ext dotenv
%dotenv
%reload_ext dotenv

Set up the LLM

In [3]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI

model="gpt-4o-mini"
llm = OpenAI(model=model)
Settings.llm = llm

Set up all components.

In [4]:
from llama_index.core import get_response_synthesizer
from llama_index.core.chat_engine import CondensePlusContextChatEngine
from llama_index.core.memory import ChatMemoryBuffer

from llama_index_cmem.retrievers.cmem.cmem_retriever import CMEMRetriever

ontology_graph = "http://ld.company.org/prod-vocab/"
context_graph = "http://ld.company.org/prod-inst/"

response_synthesizer = get_response_synthesizer()

retriever = CMEMRetriever(
    ontology_graph=ontology_graph,
    context_graph=context_graph,
    llm=llm
)

memory = ChatMemoryBuffer.from_defaults()

chat_engine = CondensePlusContextChatEngine(
    retriever=retriever,
    llm=llm,
    memory=memory
)

Define a natural language query, run via query engine and get a natural language response answering the question.

In [5]:
from IPython.display import display, Markdown

def ask(prompt):
    text = f"## Prompt: _{prompt}_\n\n"
    text += "### Final response:\n\n"
    display_handle = display(Markdown(""), display_id=True)
    display_handle.update(Markdown(text))
    response = chat_engine.stream_chat(prompt)
    for token in response.response_gen:
        text += f"{token}"
        display_handle.update(Markdown(text))

query = "List all hardware with price. Limit the results to 20 items."
ask(query)

## Prompt: _List all hardware with price. Limit the results to 20 items._

### Final response:

Here are 20 hardware items along with their prices:

1. **Hardware**: [hw-F332-3707903](http://ld.company.org/prod-instances/hw-F332-3707903) - **Price**: 2.44 EUR
2. **Hardware**: [hw-E652-6887116](http://ld.company.org/prod-instances/hw-E652-6887116) - **Price**: 2.29 EUR
3. **Hardware**: [hw-N560-4369045](http://ld.company.org/prod-instances/hw-N560-4369045) - **Price**: 2.61 EUR
4. **Hardware**: [hw-P903-7514284](http://ld.company.org/prod-instances/hw-P903-7514284) - **Price**: 5.97 EUR
5. **Hardware**: [hw-C247-3833661](http://ld.company.org/prod-instances/hw-C247-3833661) - **Price**: 3.44 EUR
6. **Hardware**: [hw-N480-3487616](http://ld.company.org/prod-instances/hw-N480-3487616) - **Price**: 0.79 EUR
7. **Hardware**: [hw-N832-1055352](http://ld.company.org/prod-instances/hw-N832-1055352) - **Price**: 3.48 EUR
8. **Hardware**: [hw-L128-5499058](http://ld.company.org/prod-instances/hw-L128-5499058) - **Price**: 0.51 EUR
9. **Hardware**: [hw-V382-3883746](http://ld.company.org/prod-instances/hw-V382-3883746) - **Price**: 3.68 EUR
10. **Hardware**: [hw-N573-1498086](http://ld.company.org/prod-instances/hw-N573-1498086) - **Price**: 5.53 EUR
11. **Hardware**: [hw-Z143-3180282](http://ld.company.org/prod-instances/hw-Z143-3180282) - **Price**: 4.31 EUR
12. **Hardware**: [hw-R946-7112528](http://ld.company.org/prod-instances/hw-R946-7112528) - **Price**: 2.86 EUR
13. **Hardware**: [hw-K926-9334427](http://ld.company.org/prod-instances/hw-K926-9334427) - **Price**: 3.13 EUR
14. **Hardware**: [hw-W658-9979899](http://ld.company.org/prod-instances/hw-W658-9979899) - **Price**: 4.90 EUR
15. **Hardware**: [hw-A750-6901242](http://ld.company.org/prod-instances/hw-A750-6901242) - **Price**: 5.49 EUR
16. **Hardware**: [hw-K313-8452822](http://ld.company.org/prod-instances/hw-K313-8452822) - **Price**: 2.92 EUR
17. **Hardware**: [hw-L816-8238278](http://ld.company.org/prod-instances/hw-L816-8238278) - **Price**: 3.21 EUR
18. **Hardware**: [hw-X407-9945990](http://ld.company.org/prod-instances/hw-X407-9945990) - **Price**: 2.36 EUR
19. **Hardware**: [hw-G249-4926490](http://ld.company.org/prod-instances/hw-G249-4926490) - **Price**: 2.44 EUR
20. **Hardware**: [hw-Z212-3451369](http://ld.company.org/prod-instances/hw-Z212-3451369) - **Price**: 5.92 EUR

Let me know if you need more information!